In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.4 MB/s eta 0:00:0000:01


In [2]:
!pip install transformers datasets huggingface_hub torch

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/sample_submission.csv
/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/train.csv
/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/test.csv
/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/cats


In [2]:
## Common ML Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import json
import os
import random
import math
import dotenv

In [5]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
import datasets
from datasets import Dataset
from sklearn.model_selection import train_test_split
from huggingface_hub import HfApi, Repository, login

In [6]:
import evaluate

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
HF_TOKEN = "<HF-Token>"

In [9]:
login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
device

device(type='cuda')

In [4]:
common_file_path = "/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/{_fn}"

In [5]:
raw_train = pd.read_csv(common_file_path.format(_fn="train.csv"))
raw_test = pd.read_csv(common_file_path.format(_fn="test.csv"))
raw_smplsub = pd.read_csv(common_file_path.format(_fn="sample_submission.csv"))

In [11]:
raw_train.iloc[1]

Unnamed: 0                                                    1
text          the learning point open digital education. a r...
target                                       academic interests
Word Count                                                  147
Name: 1, dtype: object

In [10]:
raw_train.iloc[1]['text']

'the learning point open digital education. a repository of tutorials and visualizations to help students learn computer science, mathematics, physics and electrical engineering basics. visualizations are in the form of java applets and html5 visuals. graphical educational content for mathematics, science, computer science. cs topics covered : greedy algorithms, dynamic programming, linked lists, arrays, graphs, depth first search, breadth first search, dfs and bfs, circular linked lists, functional programming, programming interview questions, graphics and solid modelling toolsphysics : projectile motion, mechanics, electrostatics, electromagnetism, engineering mechanics, optical instruments, wave motion, applets and visualizations. mathematics: algebra, linear algebra, trigonometry, euclidean and analytic geometry, probability, game theory, operations research, calculus of single/multiple variable(s). electrical engineering : dc circuits, digital circuits. a listing of cbse and cisce

In [14]:
with open(common_file_path.format(_fn="cats"), "rb") as fp:
    cats = pickle.load(fp)

In [15]:
cats

['academic interests',
 'books and literature',
 'healthy living',
 'careers',
 'news and politics',
 'shopping',
 'style and fashion',
 'family and relationships',
 'business and finance',
 'automotives',
 'pharmaceuticals, conditions, and symptoms',
 'arts and culture',
 'sports',
 'pets',
 'hobbies and interests',
 'real estate',
 'food and drinks',
 'home and garden',
 'video gaming',
 'movies',
 'travel',
 'personal finance',
 'technology and computing',
 'music and audio',
 'television',
 'health']

In [16]:
cats_dict = {}
for i in range(len(cats)):
    cats_dict[ cats[i] ] = i

In [17]:
cats_dict

{'academic interests': 0,
 'books and literature': 1,
 'healthy living': 2,
 'careers': 3,
 'news and politics': 4,
 'shopping': 5,
 'style and fashion': 6,
 'family and relationships': 7,
 'business and finance': 8,
 'automotives': 9,
 'pharmaceuticals, conditions, and symptoms': 10,
 'arts and culture': 11,
 'sports': 12,
 'pets': 13,
 'hobbies and interests': 14,
 'real estate': 15,
 'food and drinks': 16,
 'home and garden': 17,
 'video gaming': 18,
 'movies': 19,
 'travel': 20,
 'personal finance': 21,
 'technology and computing': 22,
 'music and audio': 23,
 'television': 24,
 'health': 25}

In [18]:
raw_train['label'] = raw_train.target.apply(lambda x : cats_dict[x])

In [19]:
raw_train.head(5)

,Unnamed: 0,text,target,Word Count,label
0,0,"python courses python courses, python exercise...",academic interests,125,0
1,1,the learning point open digital education. a r...,academic interests,147,0
2,2,"tech news, latest technology, mobiles, laptops...",academic interests,143,0
3,3,the best it certification materials in usa | k...,academic interests,364,0
4,4,"bioland scientific, for your research needs bi...",academic interests,176,0


In [20]:
X_main = list(raw_train["text"].values)
y_main = list(raw_train["label"].values)

In [21]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X_main, y_main, random_state=42, test_size=0.1, stratify=y_main
)

In [22]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        # Tokenize the text when it's retrieved
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}  # Squeeze to remove batch dim
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [23]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased", num_labels=26)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
test_dataset = NewsDataset(test_texts, test_labels, tokenizer)

In [25]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')  # Multi-class metric
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [29]:
# Define batch size variable
batch_size = 32  # Change this to any value you want (32, 64, etc.)

training_args = TrainingArguments(
    output_dir="results",  # output directory
    overwrite_output_dir=True,
    num_train_epochs=2,  # total number of training epochs
    per_device_train_batch_size=batch_size,  # use the batch_size variable
    per_device_eval_batch_size=batch_size,  # use the batch_size variable
    warmup_steps=500,  # reduce warmup steps to speed up training
    weight_decay=0.01,  # strength of weight decay
    logging_dir="logs",  # directory for storing logs
    logging_steps=5000,  # log after every 500 steps
    save_steps=5000,  # save checkpoint every 500 steps
    learning_rate=1e-5,  # maintain learning rate
    do_train=True,
    do_eval=True,
    seed=42,
    gradient_accumulation_steps=1,  # no need for gradient accumulation with a large batch size
    fp16=True,  # Enable mixed precision training for faster computations
    push_to_hub=True,
    hub_model_id="LakshitKava/Fibe_IAB_DB_v2",
    report_to="all"  # Reports logs to both TensorBoard and Hub
)

# Loading the model with 26 output labels
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=26)

trainer = Trainer(
    model=model,  # the instantiated Transformers model to be trained
    args=training_args,  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,  # training dataset
    eval_dataset=test_dataset,  # test dataset
)

# Optional: Use a DataLoader with more workers for faster data loading
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [30]:
model.device

device(type='cuda', index=0)

In [31]:
%%time
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
5000,1.013500
10000,0.597300
15000,0.526900


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

CPU times: user 10h 32min 58s, sys: 1min 31s, total: 10h 34min 30s
Wall time: 10h 28min 47s


TrainOutput(global_step=19618, training_loss=0.6650083240279973, metrics={'train_runtime': 37718.6128, 'train_samples_per_second': 33.287, 'train_steps_per_second': 0.52, 'total_flos': 1.6639036304176742e+17, 'train_loss': 0.6650083240279973, 'epoch': 2.0})

In [32]:
trainer

In [33]:
trainer.push_to_hub(commit_message="LakshitKava/Fibe_IAB_DB_v2_trainer")

CommitInfo(commit_url='https://huggingface.co/LakshitKava/Fibe_IAB_DB_v2/commit/10223c498666e9bfcd39000a706fa37ac6fdbe8a', commit_message='LakshitKava/Fibe_IAB_DB_v2_trainer', commit_description='', oid='10223c498666e9bfcd39000a706fa37ac6fdbe8a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/LakshitKava/Fibe_IAB_DB_v2', endpoint='https://huggingface.co', repo_type='model', repo_id='LakshitKava/Fibe_IAB_DB_v2'), pr_revision=None, pr_num=None)

In [34]:
trainer.save_model(output_dir="mdl")

No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
%%time
trainer.evaluate()